# Prophet + 23 regressor (1, 7, 30 kunlik forecast)

- Target: `rate`; `date` tashlanadi; qolgan 23 ustun regressorga beriladi.
- Outlier clip (IQR), 70/15/15 split, MAE/MAPE.
- 1/7/30 kunlik forecast: agar kelajak featurelar mavjud bo'lmasa, oxirgi qatordagi qiymatlar takrorlanadi.


In [ ]:
print('hi')

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
# Ma'lumotlarni yuklash
p = Path('datasets/usd_rates_ready.csv')
df = pd.read_csv(p)
df.columns = [c.lower() for c in df.columns]
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Target = rate, date ishlatilmaydi, qolgan 23 ta ustun feature sifatida
feature_cols = [c for c in df.columns if c not in ['date', 'rate']]

# Outlierlarni IQR bilan yumshatish (faqat targetda)
def clip_iqr(s, k=1.5):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - k * iqr, q3 + k * iqr
    return s.clip(lower, upper)

df['rate'] = clip_iqr(df['rate'])

# Vaqt bo'yicha train/val/test (70/15/15)
n = len(df)
train_end = int(n * 0.70)
val_end = int(n * 0.85)
train_df = df.iloc[:train_end]
val_df = df.iloc[train_end:val_end]
test_df = df.iloc[val_end:]

# Bashorat kunlari
horizons = [1, 7, 30]

try:
    from prophet import Prophet
except ImportError:
    raise SystemExit("Prophet o'rnatilmagan: pip install prophet")

train_p = train_df[['date', 'rate'] + feature_cols].rename(columns={'date': 'ds', 'rate': 'y'})
val_p = val_df[['date', 'rate'] + feature_cols].rename(columns={'date': 'ds', 'rate': 'y'})
test_p = test_df[['date', 'rate'] + feature_cols].rename(columns={'date': 'ds', 'rate': 'y'})

m = Prophet(daily_seasonality=False, weekly_seasonality=True, yearly_seasonality=True)
for col in feature_cols:
    m.add_regressor(col)

m.fit(train_p)

# Val/test baholash
full_future = pd.concat([train_p, val_p, test_p], ignore_index=True)
forecast = m.predict(full_future)
val_start = len(train_p)
val_end_idx = val_start + len(val_p)
test_end_idx = val_end_idx + len(test_p)
val_pred = forecast.iloc[val_start:val_end_idx]['yhat']
test_pred = forecast.iloc[val_end_idx:test_end_idx]['yhat']

mae_val = mean_absolute_error(val_p['y'], val_pred)
mape_val = mean_absolute_percentage_error(val_p['y'], val_pred)
mae_test = mean_absolute_error(test_p['y'], test_pred)
mape_test = mean_absolute_percentage_error(test_p['y'], test_pred)
print(f"Val MAE: {mae_val:.2f}, MAPE: {mape_val:.4f}")
print(f"Test MAE: {mae_test:.2f}, MAPE: {mape_test:.4f}")

# 1/7/30 kunlik forecast (regressorlarni oxirgi qatordan takrorlab to'ldiramiz)
last_feat = df[feature_cols].iloc[-1]
horizon_forecasts = {}
for h in horizons:
    future_dates = pd.date_range(df['date'].max() + pd.Timedelta(days=1), periods=h, freq='D')
    future_reg = pd.DataFrame(np.repeat([last_feat.values], h, axis=0), columns=feature_cols)
    future_df = pd.concat([pd.DataFrame({'ds': future_dates}), future_reg], axis=1)
    fc = m.predict(future_df)
    horizon_forecasts[f'h_{h}'] = fc[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# Namuna chiqishi
horizon_forecasts['h_30'].head()
